# Surveying Telescopes

Analysing data from an optical interferometer to determine the 3-dimensional locations of the telescopes.

Data are a complete year (2012) of optical path delay observations from the CHARA interferometer in California (see http://www.chara.gsu.edu).

In [1]:
import sys
import pandas as pd
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [2]:
# function to load file
def load_file(filename):
    try:
        f = open(filename, 'rb')
    except IOError:
        print('Cannot open', filename)
        sys.exit()
    with f:
        return pd.read_csv(f, index_col=0)

In [3]:
# convert day,hour,minute,second to second
def create_time_series(dataframe):
    day_column = np.zeros(((len(dataframe)),1), dtype=float)
    time_column = np.zeros(((len(dataframe)),1), dtype=float)
    
    for i in range(0,len(dataframe)-1):
        if dataframe['h'].iloc[i+1] < dataframe['h'].iloc[i]:
            day_column[i+1] = day_column[i]+1
        else:
            day_column[i+1] = day_column[i]
    
    for i in range(0,len(dataframe)):
        time_column[i] = day_column[i]*24.0*3600.0+dataframe['h'].iloc[i]*3600.0+dataframe['m'].iloc[i]*60.0+dataframe['s'].iloc[i]
    
    return dataframe.assign(day=day_column, time=time_column)

In [20]:
df = load_file('2012_S1W2.csv')
df = create_time_series(df)

In [21]:
# fit function
def fit_function(xdata, Bx, By, Bz, D_POP, A1, w1, omega1, A2, w2, omega2):
    Sx,Sy,Sz,t = xdata
    return Sx*Bx+Sy*By+Sz*Bz+D_POP+A1*np.cos(w1*t+omega1)*A2*np.cos(w2*t+omega2)

In [22]:
# fitting data
xdata = [np.array(np.cos(np.radians(df['el']))*np.sin(np.radians(df['az']))),\
         np.array(np.cos(np.radians(df['el']))*np.cos(np.radians(df['az']))),\
         np.array(np.sin(np.radians(df['el']))),\
         np.array(df['time'])]

ydata = np.array(df['delay2']-df['delay1'])

parameters, cov = optimize.curve_fit(fit_function, xdata, ydata)
print(parameters)

[-6.90950419e+01  1.99336881e+02  4.57925881e-01  8.46667131e+00
 -7.69769046e-04  1.00003735e+00  1.18342861e+00  1.60306999e+00
  9.99962858e-01  1.90169861e+00]
